In [1]:
import re
from TextCleaner import TextCleaner
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sn
import keras_metrics as km
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from numpy import sqrt 
%matplotlib inline
import nltk
from langdetect import detect
import contractions
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import PorterStemmer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU, SimpleRNN, SpatialDropout1D
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
from tensorflow.python.keras.metrics import Metric
import tensorflow as tf    
import tensorflow_addons as tfa
from tensorflow import keras 
from keras.metrics import Precision, Recall

# Classifiers
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn.naive_bayes import (
    BernoulliNB,
    ComplementNB,
    MultinomialNB,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
df = pd.read_csv('sentiment.csv')
df = df.drop(df.columns[0], axis=1)
df

Author                                               Text  \
0            yusobays  @elonmusk @elonmusk  please Come to TRY₺ #turk...   
1       BtcTurkDestek  @DrBatal  Hello, if you have registered using ...   
2      RisingValhalla  “I trade weekly bars. You?”\n\n200 day moving ...   
3        Erda55203615  @elonmusk Try turkish lira its at the bottom m...   
4        Erda55203615            @ImTooRealForU TRY FUCKING TURKISH LIRA   
...               ...                                                ...   
1944  flyingstocksman  For those of you who missed it last night with...   
1945         alpeshbp  GBP/USD Weekly Price Forecast – The British Po...   
1946     campsbay_com  20.0142 ZAR for 1 GBP @ December 31, 2020 at 0...   
1947       MajorFXNow  GBP/EUR: Pound Lifted by UK Approval of AstraZ...   
1948     trader_blitz  GBP/JPY | $GBPJPY | $GBP $JPY\n\nBRITISH POUND...   

                           Time  Like Count Currency ISO Code  \
0     2021-01-31 22:21:58+00:00           1               TRY   
1     2021-01-30 18:07:15+00:00           0               TRY   
2     2021-01-30 03:29:59+00:00           0               TRY   
3     2021-01-29 20:20:49+00:00           0               TRY   
4     2021-01-29 20:19:36+00:00           0               TRY   
...                         ...         ...               ...   
1944  2021-01-01 09:24:17+00:00           0               GBP   
1945  2021-01-01 07:13:32+00:00           0               GBP   
1946  2021-01-01 04:20:02+00:00           0               GBP   
1947  2021-01-01 02:32:32+00:00           0               GBP   
1948  2021-01-01 00:42:01+00:00           0               GBP   

                                                    Url  target  
0     https://twitter.com/yusobays/status/1356004827...     0.0  
1     https://twitter.com/BtcTurkDestek/status/13555...     0.0  
2     https://twitter.com/RisingValhalla/status/1355...     0.0  
3     https://twitter.com/Erda55203615/status/135524...    -1.0  
4     https://twitter.com/Erda55203615/status/135524...    -1.0  
...                                                 ...     ...  
1944  https://twitter.com/flyingstocksman/status/134...     0.0  
1945  https://twitter.com/alpeshbp/status/1344904575...     0.0  
1946  https://twitter.com/campsbay_com/status/134486...     0.0  
1947  https://twitter.com/MajorFXNow/status/13448338...     0.0  
1948  https://twitter.com/trader_blitz/status/134480...     0.0  

[1949 rows x 7 columns]

In [3]:
def sentiments(df):
    if df['target'] >= 0:
        return 'Positive'
    elif df['target'] < 0:
        return 'Negative'      

In [4]:
df['Sentiment'] = df.apply(sentiments, axis=1)

In [5]:
df

Author                                               Text  \
0            yusobays  @elonmusk @elonmusk  please Come to TRY₺ #turk...   
1       BtcTurkDestek  @DrBatal  Hello, if you have registered using ...   
2      RisingValhalla  “I trade weekly bars. You?”\n\n200 day moving ...   
3        Erda55203615  @elonmusk Try turkish lira its at the bottom m...   
4        Erda55203615            @ImTooRealForU TRY FUCKING TURKISH LIRA   
...               ...                                                ...   
1944  flyingstocksman  For those of you who missed it last night with...   
1945         alpeshbp  GBP/USD Weekly Price Forecast – The British Po...   
1946     campsbay_com  20.0142 ZAR for 1 GBP @ December 31, 2020 at 0...   
1947       MajorFXNow  GBP/EUR: Pound Lifted by UK Approval of AstraZ...   
1948     trader_blitz  GBP/JPY | $GBPJPY | $GBP $JPY\n\nBRITISH POUND...   

                           Time  Like Count Currency ISO Code  \
0     2021-01-31 22:21:58+00:00           1               TRY   
1     2021-01-30 18:07:15+00:00           0               TRY   
2     2021-01-30 03:29:59+00:00           0               TRY   
3     2021-01-29 20:20:49+00:00           0               TRY   
4     2021-01-29 20:19:36+00:00           0               TRY   
...                         ...         ...               ...   
1944  2021-01-01 09:24:17+00:00           0               GBP   
1945  2021-01-01 07:13:32+00:00           0               GBP   
1946  2021-01-01 04:20:02+00:00           0               GBP   
1947  2021-01-01 02:32:32+00:00           0               GBP   
1948  2021-01-01 00:42:01+00:00           0               GBP   

                                                    Url  target Sentiment  
0     https://twitter.com/yusobays/status/1356004827...     0.0  Positive  
1     https://twitter.com/BtcTurkDestek/status/13555...     0.0  Positive  
2     https://twitter.com/RisingValhalla/status/1355...     0.0  Positive  
3     https://twitter.com/Erda55203615/status/135524...    -1.0  Negative  
4     https://twitter.com/Erda55203615/status/135524...    -1.0  Negative  
...                                                 ...     ...       ...  
1944  https://twitter.com/flyingstocksman/status/134...     0.0  Positive  
1945  https://twitter.com/alpeshbp/status/1344904575...     0.0  Positive  
1946  https://twitter.com/campsbay_com/status/134486...     0.0  Positive  
1947  https://twitter.com/MajorFXNow/status/13448338...     0.0  Positive  
1948  https://twitter.com/trader_blitz/status/134480...     0.0  Positive  

[1949 rows x 8 columns]

In [6]:
data_v1 = df[['Text','Sentiment']]
data_v1.isnull().sum()

Text         0
Sentiment    0
dtype: int64

In [7]:
plt.figure(figsize=(7,8))
plt.title("Number of records of positive and negative sentiments")
plot = sn.countplot(x = 'Sentiment', data = data_v1)
for p in plot.patches:
    plot.annotate(p.get_height(),(p.get_x() + 0.2 ,p.get_height()+80))

In [8]:
clean = TextCleaner()
text = clean.convert_emoji_to_text("Hilarious 😂. The feeling of making a sale 😎, The feeling of actually fulfilling orders 😒")
print(text)

text4 = "Hello :-) :-)"
resp = clean.convert_emoticons(text4)
print(resp)

Hilarious face with tears of joy. The feeling of making a sale smiling face with sunglasses, The feeling of actually fulfilling orders unamused face
Hello Happy face smiley Happy face smiley


In [9]:
df

Author                                               Text  \
0            yusobays  @elonmusk @elonmusk  please Come to TRY₺ #turk...   
1       BtcTurkDestek  @DrBatal  Hello, if you have registered using ...   
2      RisingValhalla  “I trade weekly bars. You?”\n\n200 day moving ...   
3        Erda55203615  @elonmusk Try turkish lira its at the bottom m...   
4        Erda55203615            @ImTooRealForU TRY FUCKING TURKISH LIRA   
...               ...                                                ...   
1944  flyingstocksman  For those of you who missed it last night with...   
1945         alpeshbp  GBP/USD Weekly Price Forecast – The British Po...   
1946     campsbay_com  20.0142 ZAR for 1 GBP @ December 31, 2020 at 0...   
1947       MajorFXNow  GBP/EUR: Pound Lifted by UK Approval of AstraZ...   
1948     trader_blitz  GBP/JPY | $GBPJPY | $GBP $JPY\n\nBRITISH POUND...   

                           Time  Like Count Currency ISO Code  \
0     2021-01-31 22:21:58+00:00           1               TRY   
1     2021-01-30 18:07:15+00:00           0               TRY   
2     2021-01-30 03:29:59+00:00           0               TRY   
3     2021-01-29 20:20:49+00:00           0               TRY   
4     2021-01-29 20:19:36+00:00           0               TRY   
...                         ...         ...               ...   
1944  2021-01-01 09:24:17+00:00           0               GBP   
1945  2021-01-01 07:13:32+00:00           0               GBP   
1946  2021-01-01 04:20:02+00:00           0               GBP   
1947  2021-01-01 02:32:32+00:00           0               GBP   
1948  2021-01-01 00:42:01+00:00           0               GBP   

                                                    Url  target Sentiment  
0     https://twitter.com/yusobays/status/1356004827...     0.0  Positive  
1     https://twitter.com/BtcTurkDestek/status/13555...     0.0  Positive  
2     https://twitter.com/RisingValhalla/status/1355...     0.0  Positive  
3     https://twitter.com/Erda55203615/status/135524...    -1.0  Negative  
4     https://twitter.com/Erda55203615/status/135524...    -1.0  Negative  
...                                                 ...     ...       ...  
1944  https://twitter.com/flyingstocksman/status/134...     0.0  Positive  
1945  https://twitter.com/alpeshbp/status/1344904575...     0.0  Positive  
1946  https://twitter.com/campsbay_com/status/134486...     0.0  Positive  
1947  https://twitter.com/MajorFXNow/status/13448338...     0.0  Positive  
1948  https://twitter.com/trader_blitz/status/134480...     0.0  Positive  

[1949 rows x 8 columns]

In [10]:
def remove_url(text):
    result = re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)
    return result

def replace_contractions(text):
    return contractions.fix(text)

abbs =  {"eur": "euro", "usd": "dollar", "gbp" : "pound", "u":"you", "tryeur":"turkish lira euro", "try":" turkish lira", "gt":"greater than","lt": "less than","btc":"bitcoin",
        "gbpusd": "pound dollar", "usdtry":"dollar turkish lira", "fx":"forex", "u":"you", "eurusd":"euro usd", "eurgbp":"euro pound", "jpy":"japanese yen","cny":"chinese yuan","uk":"united kingdom"}

def replace_currency(text):
    val = ''
    for i in text.split():
        if abbs.get(i.lower().strip()) is not None:
            #print(abbs.get(i.lower().strip()))
            val = val + str(abbs.get(i.lower().strip())) + ' '
        else:
            val = val + i + ' '
    return val

In [11]:
print(remove_url(df['Text'].iloc[1942]))

Visit  to check the latest #bitcoin price in #gbp #uk #pound. You can also use our free $GBP to $BTC price converter.


In [12]:
print(replace_currency("@elonmusk Try turkish lira its at the bottom"))

@elonmusk  turkish lira turkish lira its at the bottom 


In [13]:
clean = TextCleaner()

def cleaning(df_cl, stop_words):
    
    # Removing URLs
    df_cl["Text"] = df_cl["Text"].apply(lambda x: remove_url(x))
    
     # Removing mentioned author names
    df_cl["Text"] = df_cl["Text"].apply(lambda x: re.sub("@[A-Za-z0-9]+","", x))
    
    # Lowercase
    df_cl['Text'] = df_cl['Text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    
    df_cl["Text"] = df_cl["Text"].apply(lambda x: replace_contractions(x))
    
    df_cl["Text"] = df_cl["Text"].apply(lambda x: replace_currency(x))
    
     # Remove Non -letters
    df_cl['Text'] = df_cl['Text'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]", ' ', x))
    
    # Removing Punctuation
    df_cl["Text"] = df_cl["Text"].apply(lambda x: re.sub(r'[^\w\s]','', x))
    
    # Removing stop words
    df_cl['Text'] = df_cl['Text'].apply(lambda x:' '.join(x for x in x.split() if x not in stop_words))
    
    # Lemmatization
    df_cl['Text'] = df_cl['Text'].apply(lambda x:' '.join([Word(x).lemmatize() for x in x.split()]))
    
    # Emoticons
    df_cl['Text'] = df_cl['Text'].apply(lambda x: clean.convert_emoji_to_text(x))
    
    # Emojis
    df_cl['Text'] = df_cl['Text'].apply(lambda x: clean.convert_emoji_to_text(x))    
    
    # Stemming
    #df_cl['Text'] = df_cl['Text'].apply(lambda x: port_stemmer(x))
    return df_cl

stop_words = stopwords.words('english')

In [14]:
data_v1 = cleaning(data_v1, stop_words)

In [15]:
data_v1

Text Sentiment
0                           please come try turkishlira  Positive
1     hello registered using turkish citizenship id ...  Positive
2     trade weekly bar 200 day moving average breach...  Positive
3       turkish lira turkish lira bottom maybe would go  Negative
4                     turkish lira fucking turkish lira  Negative
...                                                 ...       ...
1944  missed last night nye potentially important tw...  Positive
1945  gbp usd weekly price forecast british pound lo...  Positive
1946  20 0142 zar 1 pound december 31 2020 06 20am s...  Positive
1947  gbp eur pound lifted united kingdom approval a...  Positive
1948  gbp jpy gbpjpy gbp jpy british pound brexit br...  Positive

[1949 rows x 2 columns]

In [16]:
common_words = ''
for i in data_v1.Text:
    i = str(i)
    tokens = i.split()
    common_words += " ".join(tokens)+" "
wordcloud = WordCloud(width=1600, height=800).generate(common_words)
plt.figure( figsize=(20,10), facecolor='k')
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [17]:
lb = LabelEncoder()
data_v1['Sentiment'] = lb.fit_transform(data_v1['Sentiment'])

In [18]:
tokenizer = Tokenizer(num_words=1000, split=' ', lower=True) 
tokenizer.fit_on_texts(data_v1['Text'].values)
X = tokenizer.texts_to_sequences(df['Text'].values)
X = pad_sequences(X)

In [19]:
y = pd.get_dummies(data_v1['Sentiment'])

In [20]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

<h3>Time Series Methodologies</h3>
<h4>Defining Models</h4>

In [21]:
models = [ 'SimpleRNN', 'LSTM', 'GRU']

In [22]:
METRICS = [ 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      km.binary_precision(), 
      km.binary_recall(),
      'mse', 
      'mae', 
      keras.metrics.AUC(name='auc'),
      tfa.metrics.F1Score(num_classes=2, threshold=0.5, average='macro'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn')
]

2022-03-06 21:32:07.008254: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
def build_model(nb_words, _model="SimpleRNN", embedding_matrix=None):
    model = Sequential()
    model.add(Embedding(input_dim=1000, output_dim=64, input_length = nb_words))
    if _model == "SimpleRNN":
        model.add(SimpleRNN(128, dropout=0.1, recurrent_dropout=0.1, activation='relu'))
    elif _model == "LSTM":
        model.add(LSTM(100, return_sequences=True))
        model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.1, activation='relu'))
    elif _model == "GRU":
        model.add(GRU(128))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[METRICS])
    return model

In [24]:
print(X.shape)
print(y.shape)

(1949, 38)
(1949, 2)


In [ ]:
output_df = pd.DataFrame(columns=['model', 'accuracy', 'precision', 'recall', 'mse', 'mae', 'auc', 'f1_score', 'prediction time'])

seed = 5
kfold = KFold(seed, True, 7)
cvscores = []

batch_size = 64

x1=np.array(X)
y1=np.array(y)

for idx, mdl in enumerate(models):
    accuracy = precision = recall = mse = mae = auc = f1_score_val = prediction_time = 0
    
    print("Model : " + str(mdl))
    model_ = build_model(X.shape[1], mdl, None)
    print("Model Summary")
    print(model_.summary())
    
    for train, test in kfold.split(x1,y1):
        X_train, X_test, y_train, y_test = train_test_split(x1, y1, test_size = 0.2, random_state = 0)
        model_.fit(X_train, y_train, epochs=5, validation_split=0.2, batch_size=batch_size, verbose = 'auto', shuffle=False, validation_data=(X_test,y_test))
    
        start_time = time.time()
        y_pred = model_.predict(X_test)
        end_time = time.time() - start_time
        print("")

        scores = model_.evaluate(X_test, y_test)
        print("%s: %.2f%%" % (model_.metrics_names[1], scores[1]*100))    
        print("")
        accuracy += scores[1]
        precision += scores[2]
        recall += scores[3]
        mse += scores[4]
        mae += scores[5]
        auc += scores[6]
        f1_score_val += scores[7]
        prediction_time += end_time
        
        #print("")
        #print("Time taken to predict the model " + str(end_time))

    output_df = output_df.append([{'model': str(mdl), 'accuracy':accuracy/seed,'precision':precision/seed
                                       ,'recall':recall/seed,'mse':mse/seed,'mae':mae/seed
                                       ,'auc':auc/seed,'f1_score':f1_score_val/seed, 'prediction time': prediction_time/seed}], ignore_index=True)

    yhat = model_.predict_on_batch(X_test)
    #print("yhat[:,1]",yhat[:,1])
    #print("y_test",y_test)
    fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test[:,1], yhat[:,1])
    plt.rcParams["figure.figsize"] = (10,7)
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_rf, tpr_rf, label=str(mdl) + ' (area = {:.3f})'.format(auc/seed))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.savefig('tsa_roc_curve.png')

    #print(classification_report(y_test.values.argmax(axis=1), y_pred.argmax(axis=1)))
    print("___________________________________________________________________________________")

    print("")
    print("")
    print("")
    
plt.show()

Model : SimpleRNN
Model Summary
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 38, 64)            64000     
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               24704     
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 88,962
Trainable params: 88,962
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


/opt/anaconda3/lib/python3.9/site-packages/keras_metrics/metrics.py:26: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return object.__getattribute__(self, name)


20/20 [==============================] - 3s 44ms/step - loss: 0.6400 - accuracy: 0.6680 - precision: 0.2500 - recall: 0.0167 - mse: 0.2237 - mae: 0.4445 - auc: 0.6863 - f1_score: 0.4051 - tp: 833.0000 - fp: 414.0000 - tn: 833.0000 - fn: 414.0000 - val_loss: 0.6419 - val_accuracy: 0.6603 - val_precision: 0.2500 - val_recall: 0.0042 - val_mse: 0.2248 - val_mae: 0.4566 - val_auc: 0.6750 - val_f1_score: 0.3977 - val_tp: 206.0000 - val_fp: 106.0000 - val_tn: 206.0000 - val_fn: 106.0000
Epoch 2/5
20/20 [==============================] - 0s 13ms/step - loss: 0.6348 - accuracy: 0.6712 - precision: 0.2500 - recall: 0.0028 - mse: 0.2214 - mae: 0.4492 - auc: 0.6818 - f1_score: 0.4016 - tp: 837.0000 - fp: 410.0000 - tn: 837.0000 - fn: 410.0000 - val_loss: 0.6451 - val_accuracy: 0.6603 - val_precision: 0.2500 - val_recall: 0.0020 - val_mse: 0.2262 - val_mae: 0.4355 - val_auc: 0.6728 - val_f1_score: 0.3977 - val_tp: 206.0000 - val_fp: 106.0000 - val_tn: 206.0000 - val_fn: 106.0000
Epoch 3/5
20/20 [=

Epoch 4/5
20/20 [==============================] - 0s 14ms/step - loss: 0.6205 - accuracy: 0.6881 - precision: 0.6425 - recall: 0.0198 - mse: 0.2147 - mae: 0.4305 - auc: 0.7093 - f1_score: 0.4661 - tp: 858.0000 - fp: 389.0000 - tn: 858.0000 - fn: 389.0000 - val_loss: 0.6509 - val_accuracy: 0.6474 - val_precision: 0.6509 - val_recall: 0.0210 - val_mse: 0.2285 - val_mae: 0.4396 - val_auc: 0.6659 - val_f1_score: 0.4097 - val_tp: 202.0000 - val_fp: 110.0000 - val_tn: 202.0000 - val_fn: 110.0000
Epoch 5/5
20/20 [==============================] - 0s 13ms/step - loss: 0.6151 - accuracy: 0.6881 - precision: 0.6525 - recall: 0.0224 - mse: 0.2122 - mae: 0.4270 - auc: 0.7119 - f1_score: 0.4717 - tp: 858.0000 - fp: 389.0000 - tn: 858.0000 - fn: 389.0000 - val_loss: 0.6552 - val_accuracy: 0.6442 - val_precision: 0.6540 - val_recall: 0.0237 - val_mse: 0.2300 - val_mae: 0.4355 - val_auc: 0.6662 - val_f1_score: 0.4084 - val_tp: 201.0000 - val_fp: 111.0000 - val_tn: 201.0000 - val_fn: 111.0000

13/13 [

/opt/anaconda3/lib/python3.9/site-packages/keras_metrics/metrics.py:26: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return object.__getattribute__(self, name)


20/20 [==============================] - 6s 131ms/step - loss: 0.6428 - accuracy: 0.6591 - precision: 0.6402 - recall: 0.0650 - mse: 0.2250 - mae: 0.4467 - auc: 0.6858 - f1_score: 0.4449 - tp: 1079.0000 - fp: 558.0000 - tn: 1079.0000 - fn: 558.0000 - val_loss: 0.6442 - val_accuracy: 0.6603 - val_precision: 0.6402 - val_recall: 0.0639 - val_mse: 0.2259 - val_mae: 0.4616 - val_auc: 0.6707 - val_f1_score: 0.3977 - val_tp: 206.0000 - val_fp: 106.0000 - val_tn: 206.0000 - val_fn: 106.0000
Epoch 2/5
20/20 [==============================] - 2s 105ms/step - loss: 0.6373 - accuracy: 0.6712 - precision: 0.6402 - recall: 0.0627 - mse: 0.2225 - mae: 0.4515 - auc: 0.6779 - f1_score: 0.4016 - tp: 837.0000 - fp: 410.0000 - tn: 837.0000 - fn: 410.0000 - val_loss: 0.6468 - val_accuracy: 0.6603 - val_precision: 0.6402 - val_recall: 0.0616 - val_mse: 0.2268 - val_mae: 0.4319 - val_auc: 0.6694 - val_f1_score: 0.3977 - val_tp: 206.0000 - val_fp: 106.0000 - val_tn: 206.0000 - val_fn: 106.0000
Epoch 3/5
20/2

20/20 [==============================] - 2s 119ms/step - loss: 0.6313 - accuracy: 0.6712 - precision: 0.6332 - recall: 0.0452 - mse: 0.2197 - mae: 0.4388 - auc: 0.6851 - f1_score: 0.4108 - tp: 837.0000 - fp: 410.0000 - tn: 837.0000 - fn: 410.0000 - val_loss: 0.6460 - val_accuracy: 0.6506 - val_precision: 0.6322 - val_recall: 0.0448 - val_mse: 0.2268 - val_mae: 0.4414 - val_auc: 0.6745 - val_f1_score: 0.3942 - val_tp: 203.0000 - val_fp: 109.0000 - val_tn: 203.0000 - val_fn: 109.0000
Epoch 4/5
20/20 [==============================] - 3s 139ms/step - loss: 0.6304 - accuracy: 0.6712 - precision: 0.6312 - recall: 0.0443 - mse: 0.2194 - mae: 0.4382 - auc: 0.6891 - f1_score: 0.4108 - tp: 837.0000 - fp: 410.0000 - tn: 837.0000 - fn: 410.0000 - val_loss: 0.6461 - val_accuracy: 0.6506 - val_precision: 0.6302 - val_recall: 0.0439 - val_mse: 0.2269 - val_mae: 0.4415 - val_auc: 0.6776 - val_f1_score: 0.3942 - val_tp: 203.0000 - val_fp: 109.0000 - val_tn: 203.0000 - val_fn: 109.0000
Epoch 5/5
20/20 

In [ ]:
output_df

<h3>Traditional Methodologies</h3>

In [ ]:
stemmer = PorterStemmer()
analyzer = CountVectorizer().build_analyzer()
def stemmed_words(doc):
    stemmed_words = [stemmer.stem(w) for w in analyzer(doc)]
    non_stop_words = [word for word in stemmed_words]
    return non_stop_words

In [ ]:
count_vectorizer = CountVectorizer(stop_words= stop_words, max_features= 1000)
feature_vector = count_vectorizer.fit(data_v1['Text'])
train_ds_features = count_vectorizer.transform(data_v1['Text'])
features = feature_vector.get_feature_names()
features_counts = np.sum(train_ds_features.toarray(), axis = 0)
features_counts = pd.DataFrame(dict(features =  features, counts = features_counts))
features_counts = features_counts.sort_values("counts", ascending= False)[0:15]
features_counts

In [ ]:
train_x, test_x, train_y, test_y =  train_test_split(train_ds_features, data_v1['Sentiment'], test_size = 0.3, random_state = 42)

In [ ]:
x = train_ds_features
y = data_v1['Sentiment']

In [ ]:
classifiers = {
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "MultinomialNB": MultinomialNB(),
    "LogisticRegression": LogisticRegression(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "MLPClassifier": MLPClassifier(max_iter=1000),
    "AdaBoostClassifier": AdaBoostClassifier()
}

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

for name, sklearn_classifier in classifiers.items():
    print("Classifier : ", name)
    accuracy = precision = recall = mse = mae = auc = f1_score_val = prediction_time = 0
    
    for train_index, test_index in kfold.split(x):
        train_x, test_x, train_y, test_y = x[train_index], x[test_index], y[train_index], y[test_index]       
    
        sklearn_classifier.fit(train_x.toarray(), train_y)
        start_time = time.time()
        test_ds_predicted = sklearn_classifier.predict(test_x.toarray())
        end_time = time.time() - start_time
        print(metrics.classification_report(test_y, test_ds_predicted))
        print("")
        print("Time taken to predict the model " + str(end_time))
        print("")
        tr_mae = mean_absolute_error(test_y, test_ds_predicted)
        tr_mse = mean_squared_error(test_y, test_ds_predicted)
        tr_accuracy = accuracy_score(test_y, test_ds_predicted)
        tr_f1_score = f1_score(test_y, test_ds_predicted, average='weighted')
        tr_recall = recall_score(test_y, test_ds_predicted, average='weighted')
        tr_precision = precision_score(test_y, test_ds_predicted, average='weighted')
        tr_auc = roc_auc_score(test_y, test_ds_predicted)

        accuracy += tr_accuracy
        precision += tr_precision
        recall += tr_recall
        mse += tr_mse
        mae += tr_mae
        auc += tr_auc
        f1_score_val += tr_f1_score
        prediction_time += end_time
    #print('MAE: %f' % mean_absolute_error(test_y, test_ds_predicted))
    #print('MSE: %f' % mean_squared_error(test_y, test_ds_predicted))
    #print('RMSE: %f' % sqrt(mean_squared_error(test_y, test_ds_predicted)))
    
    
    output_df = output_df.append([{'model': str(name), 'accuracy':accuracy/seed,'precision':precision/seed
                                       ,'recall':recall/seed,'mse':mse/seed,'mae':mae/seed
                                       ,'auc':auc/seed,'f1_score':f1_score_val/seed, 'prediction time': prediction_time/seed}], ignore_index=True)

    yhat = sklearn_classifier.predict_proba(test_x.toarray())
    fpr_rf, tpr_rf, threshold2 = roc_curve(test_y.to_numpy(), yhat[:,1])
    plt.rcParams["figure.figsize"] = (10,7)
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_rf, tpr_rf, label=str(name) + ' (area = {:.3f})'.format(tr_auc))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.savefig('td_roc_curve.png')
    
    print("________________________________________________________")
    print("")
    print("")
plt.show()

In [ ]:
output_df

In [ ]:
output_df.to_csv('classifiers_output_result.csv', sep='\t')